In [1]:
import numpy as np
import torch, torchaudio
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import torch, os, bisect, json
from pathlib import Path

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torchaudio")

# Add this at the beginning of your notebook:
import torch._inductor.config as config
config.triton.cudagraphs = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

if torch.cuda.is_available():
    major, minor = torch.cuda.get_device_capability()
    print(f"Compute Capability: {major}.{minor}")
    if major >= 8:
        print("✅ TF32 is supported (Ampere or newer).")
    else:
        print("❌ TF32 is not supported.")

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'  # For better debugging
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True
scaler = torch.amp.GradScaler('cuda', enabled=True)

Using device: cuda
Compute Capability: 8.6
✅ TF32 is supported (Ampere or newer).
